In [0]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from requests import *
import json

In [0]:
client = dataiku.api_client()

In [0]:
dss_version = dataiku.get_custom_variables()['dss_version']

params = dict(
    meta = ['label', 'description', 'author', 'url', 'tags', 'supportLevel'],
    default = ['storeVersion','downloadURL', 'size', 'content']
)

In [0]:
url = "https://update.dataiku.com/dss/{}/plugins/list.json".format(dss_version)

In [0]:
page_request = json.loads(request('GET', url).content)['items']

In [0]:
def get_plugin_meta(item, params):

    plugin_meta = {}

    for k,v in params.items():

        if k=='default':
            for e in v:
                if e in item.keys():
                    plugin_meta[e] = item[e]
                else:
                    plugin_meta[e] = np.nan
        else:
            for e in v:
                if k in item.keys() and e in item[k].keys():
                    plugin_meta[e] = item[k][e]
                else:
                    plugin_meta[e] = np.nan
    return plugin_meta

items = {}

for i, item in enumerate(page_request):

    plugin_meta = get_plugin_meta(item, params)
    plugin_meta['release_time']= max([item['revisions'][i]['releaseTime'] for  i in range(len(item['revisions']))])

    items[item['id']] = plugin_meta

df_plugins = pd.DataFrame.from_dict(items,orient='index').reset_index().rename(columns={'index':'pluginId'})

In [0]:
df_plugins

In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

plugins_store_df = ... # Compute a Pandas dataframe to write into plugins_store


# Write recipe outputs
plugins_store = dataiku.Dataset("plugins_store")
plugins_store.write_with_schema(plugins_store_df)